Železniške povezave so podane v obliki slovarja. Ključi so imena postaj (npr. Zidani most), pripadajoče vrednosti pa seznami povezav na neposredno povezane postaje. Povezava je trojka z imenom ciljne postaje (npr. Laško), časom odhoda (v minutah od polnoči) s postaje v ključu (Zidani most) in časom prihoda na ciljno postajo (npr. Laško).

S `povezave["Zidani Most"]`, recimo, dobimo seznam vseh povezav iz Zidanega Mosta. Ta je lahko, na primer, takšen: `[('Laško', 364, 382), ('Hrastnik', 365, 386), ('Laško', 375, 390), ('Radeče', 379, 395)]`. Iz tega razberemo, da iz Zidanega Mostu odhaja vlak v Laško ob 6:04 in prispe tja ob 6:22, nato v Hrastnik ob 6:05 s prihodom ob 6:26, potem spet v Laško ob 6:15 s prihodom ob 6:30, potem v Radeče ob 6:19 s prihodom ob 6:35. **Povezave so urejene po časih odhodov.**

Časi so izmišljeni in vsaka povezava z resničnimi prihodi in odhodi vlakov je zgolj naključna. V tem smislu so podobni dejanskim voznim redom Slovenskih železnic.

## 1. Začetek obratovanja

Napišite funkcijo `prvi_vlak(povezave, postaja)`, ki prejme slovar s postajami in ime postaje. Vrniti mora čas, ko odide prvi vlak s te postaje. Klic `prvi_vlak(povezave, "Zidani Most")` bi v gornjem primeru vrnil `364`.

### Rešitev

S `povezave[postaja]` dobimo seznam vseh odhodov s te postaje. Zanima nas prvi odhod; ker so povezave (kot piše v uvodu) urejene po časih odhodov, je to kar prvi element tega seznama, torej `povezave[postaja][0]`. Tako dobimo trojko s tremi elementi: prvi pove, kam gre vlak, drugi kdaj in tretji čas prihoda na ciljno postajo. Zanima nas drugi element, torej tisti z indeksom 1. Rešitev je torej:

In [1]:
def prvi_vlak(povezave, postaja):
    return povezave[postaja][0][1]

## 2. Iskanje povezav

Napišite funkcijo `naslednja_povezava(povezave, odkod, kam, cas)`, ki poišče prvi vlak, ki odide iz `odkod` v `kam` ob podanem času `cas` ali po njem. Funkcije naj vrne čas, ko bo ta vlak prispel v kam. Če ob tem času (oziroma kasneje) ni več nobenega vlaka iz odkod v kam, ali pa te povezave sploh ni, funkcije vrne `None`. Klic `naslednja_povezava(povezave, "Zidani Most", "Laško", 362)` vrne `390`.

### Rešitev

Spet nas zanimajo vsi odhodi s podane postaje, torej `povezave[odkod]`. Gremo čez seznam trojk; njene elemente bomo shranili v spremenljivke `k`, `odhod`, `prihod`. Čim naletimo na trojko, ki gre, kamor hočemo (`k == kam`) in odide ob podanem času ali kasneje (odhod >= cas), vrnemo čas prihoda.

Če se zanka izteče, ne da bi karkoli vrnili, vrnemo `None`.

In [2]:
def naslednja_povezava(povezave, odkod, kam, cas):
    for k, odhod, prihod in povezave[odkod]:
        if k == kam and odhod >= cas:
            return prihod
    return None

## 2. Postaje

Napišite funkcijo koncna_postaja(povezave, postaja), ki prejme povezave in ime postaje. Funkcija vrne True, če vlaki s te postaje vozijo na eno samo drugo postajo in False, če vozijo na več različnih postaj.

Napišite funkcijo koncne_postaje(povezave), ki prejme slovar s povezavami in vrne seznam imen vseh končnih postaj. 

### Rešitev

Najprej sestavimo seznam vseh postaj, na katere vozijo vlaki s podobne postaje. Če je dolžina tega seznama `1`, vrnemo `True`, sicer `False`.

In [3]:
def koncna_postaja(povezave, postaja):
    povezane = []
    for k, _, _ in povezave[postaja]:
        if k not in povezane:
            povezane.append(k)
    return len(povezane) == 1

V drugi nalogi gremo čez vse ključe slovarja `povezave`. Za vsakega pokličemo prvo funkcijo in če ta vrne True, damo ime postaje v seznam, ki ga na koncu vrnemo.

In [4]:
def koncne_postaje(povezave):
    koncne = []
    for k in povezave:
        if koncna_postaja(povezave, k):
            koncne.append(k)
    return koncne

## 4. Rentabilnost

Naslednje funkcije bodo prejele dve numpyjevi tabeli: `postaje` je tabela z imeni postaj, `potniki[i, j]` pa vsebuje število potnikov, ki se je na določen dan prepeljalo s `postaje[i]` na `postaje[j]`. Napišite naslednje funkcije.

- `naj_odhodov(postaje, potniki)` vrne ime postaje, s katere se je odpeljalo največ potnikov.
- `naj_prihodov(postaje, potniki)` vrne ime postaje, na katero je prispelo največ potnikov.
- `naj_postaje(postaje, potniki)` vrne podoben rezultat kot prejšnja funkcija, le da namesto ene same postaje vrne tabelo z vsemi postajami, vendar urejeno padajoče, od postaj z največ prihodi do tiste z najmanj.

### Rešitev

Tole je bila naloga iz osi v numpyju in iz kombiniranja argmax z indeksiranjem. Prvi dve funkciji sta bili res kratki.

In [9]:
def naj_odhodov(postaje, potniki):
    return postaje[np.argmax(np.sum(potniki, axis=1))]

def naj_prihodov(postaje, potniki):
    return postaje[np.argmax(np.sum(potniki, axis=0))]

Za drugo pa je bilo potrebno sešteti število potnikov na enak način kot v prejšnji, vendar namesto `argmax` uporabiti `argsort`, da dobimo vrstni red indeksov; z [::-1] obrnemo njihov vrstni red. Nato vrnemo postaje v takšnem vrstnem redu.

In [5]:
def naj_postaje(imena_postaj, potniki):
    promet = np.sum(potniki, axis=0)
    indeks = np.argsort(promet)[::-1]
    return imena_postaj[indeks]

Seveda gre tudi to v eni vrstici, vendar bodimo raje pregledni kot kratki.

## 5. Zmanjšanje gneče

Slovenske železnice bi rade povečale udobje potnikov. V ta namen nameravajo zmanjšati gnečo na vlakih, ki je, kakor vemo, pretežno strašna. Le kdo si ne bi želel potovati na delo ali predavanje z vlakom!

Vsak potnik se bo lahko v določenem dnevu smel peljati le določeno število povezav daleč od postaje, v kateri je začel pot. Napišite funkcijo `dosegljivo(povezave, zacetna, korakov)`, ki prejme povezave, ime izhodiščne postaje in število dovoljenih korakov. Vrniti mora množico ali seznam vseh postaj, ki jih lahko doseže v tem dnevu. Če bo vaša funkcija vrnila seznam, je zaželeno, da se vsaka postaja pojavi le enkrat.

Opozorilo: funkcijo bodo poklicali testi. Ne kličite je sami, ker se bo ob nerodnem klicu mogoče izvajala zelooooo dolgo in boste imeli sitnosti.

### Rešitev

V začetku je dosegljiva podana začetna postaja. Če je dovoljen še kakšen korak, dodamo v to množico vse postaje, ki so dosegljive s te postaje, vendar dovolimo en korak manj (saj smo enega naredili s tem klicem).

In [11]:
def dosegljivo(povezave, zacetek, korakov):
    dosegljive = {zacetek}
    if korakov > 0:
        for postaja, _, _ in povezave[zacetek]:
            dosegljive |= dosegljivo(povezave, postaja, korakov - 1)
    return dosegljive